In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Problem 1 Version 1

In [6]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.8
m.cap = 15
m.dispCost = 1.2

# Define implicit variables
m.MarkPrice = Var(m.ITEMS, domain = NonNegativeReals)
m.Demand = Var(m.ITEMS, domain = NonNegativeReals)
m.Revenue = Var(m.ITEMS, domain = NonNegativeReals)
m.FinalInv = Var(m.ITEMS, domain = NonNegativeReals)

def Define_MarkPrice_Rule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i] == m.MarkPrice[i]

def Define_Demand_Rule(m, i):
    return m.MarkPct[i] * m.initInv[i] == m.Demand[i]

def Define_Revenue_Rule(m, i):
    return m.MarkPct[i] * m.Demand[i] == m.Revenue[i]

def Define_FinalInv(m, i):
    return m.initInv[i] * m.Demand[i] == m.FinalInv[i]

def Max_Discount_Rule(m, i):
    return m.MarkPct[i] <= m.maxPct

def Leftover_Inv_Rule(m, i):
    return m.FinalInv[i] <= m.cap

m.Define_MarkPrice = Constraint(m.ITEMS, rule = Define_MarkPrice_Rule)
m.Define_Demand = Constraint(m.ITEMS, rule = Define_Demand_Rule)
m.Define_Revenue = Constraint(m.ITEMS, rule = Define_Revenue_Rule)
m.Define_FinalInv = Constraint(m.ITEMS, rule = Define_FinalInv)
m.Max_Dsicount_Rule = Constraint(m.ITEMS, rule = Max_Discount_Rule)